In [1]:
import networkx as nx
import numpy as np
import pandas as pd

In [2]:
ppi = pd.read_csv('./all_interaction_dir.csv')
ppi['weighted'] = 1
dirs = pd.Series(index=(ppi['Entrez_A'].astype(str)+':'+ppi['Entrez_B'].astype(str)).values,data=ppi['Direction'].values)

G = nx.DiGraph()
G.add_weighted_edges_from(np.array(ppi[['Entrez_A', 'Entrez_B', 'weighted']].values))
cutoff = 1/len(G.nodes)

drug_info = pd.read_csv('./drug_targets.csv')
drug_targets = []
for drug in drug_info['entrez_id'].unique():
    if drug in G:
        drug_targets.append(int(drug))

In [3]:
out_cols = ['source','target','prob','path','direction']
with open('./rwr_res.csv','w') as f:
    f.write(','.join(out_cols)+'\n')

for seed in drug_targets:
    pagerank = nx.pagerank_scipy(G, alpha=0.3, personalization={seed: 1})
    pagerank = pd.Series(pagerank)
    pagerank = pagerank.sort_values(ascending=False)
    pagerank = pagerank[(pagerank>cutoff)&(pagerank.index!=seed)]

    pagerank_dir = pd.Series(index=pagerank.index,dtype=int)
    pagerank_path = pd.Series(index=pagerank.index,dtype=str)
    for target in pagerank_dir.index:
        shortest_path = nx.shortest_path(G, source=seed, target=target)
        direction = 1
        path = str(seed)
        for i,a in enumerate(shortest_path[:-1]):
            direction = (dirs[f'{a}:{shortest_path[i+1]}'])*direction
            if dirs[f'{a}:{shortest_path[i+1]}'] == 1:
                path += f'->{shortest_path[i+1]}'
            else:
                path += f'-|{shortest_path[i+1]}'
        pagerank_dir[target] = direction
        pagerank_path[target] = path

    res = pd.DataFrame(index=pagerank.index,columns=out_cols)
    res['source'] = seed
    res['target'] = pagerank.index
    res['prob'] = pagerank
    res['path'] = pagerank_path
    res['direction'] = pagerank_dir
    res.to_csv('./rwr_res.csv',mode='a',index=False,header=False)